In [ ]:
# conda activate nlp
# conda install ipykernel
# ipython kernel install --name nlp --user 

import sys
import os

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib
import requests
import csv, pandas, re, numpy

# from fake_useragent import UserAgent
# ua = UserAgent()
# hdr = {'User-Agent': ua.random,
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [ ]:
baseurl = "https://bfriends.brigitte.de/foren/coronavirus/536653-kinder-familie-kita-schule-in-zeiten-von-corona-10.html"
req = Request(baseurl, data = None, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)

usernames = [i.getText() for i in soup.find_all(["a", "span"], {"class": "username"})]
dates = [i.find(text=True, recursive = False) for i in soup.find_all("span", {"class": "date"})]
times = [i.find('span', {"class": "time"}).getText() for i in soup.find_all("div", {"class": "posthead"})]

meta = pandas.DataFrame({'user': usernames, 'date': dates, 'time': times})

In [ ]:
post = []

for i in soup.find_all('div', class_=('content')):
    ix_ = i.find(id = re.compile('post_message'))
    ix = None
    if (ix_ is not None):
        ix = ix_.get('id')
    # block quoted text
    j = i.find_all('div', class_="bbcode_container") # can quote more than one ms

    u, m = None, None
    if j:
        u = [get_quote_user(q) for q in j] # some does not have quoted user!!!
        m = [get_quote_msg(q) for q in j]
        [q.decompose() for q in j]

    ms = i.getText(strip = True)
    # emoji
    emj = None
    im = i.find('img')
    if (im is not None):
        emj = im.get('title')
    post.append([ix] + [ms] + [emj] + [u] + [m])
meta.join(pandas.DataFrame(post, columns = ['mid', 'text', 'emoji', 'user_quoted', 'quoted_text']))

In [ ]:
def get_quote_user(x):
    y = x.find('div', class_='bbcode_postedby')
    if (y is not None):
        return(y.find('strong').text)
    else:
        return("None")
    
def get_quote_msg(x):
    y = x.find('div', class_="message")
    z = x.find('div', class_="quote_container")
    if (y is not None):
        return(y.getText(strip = True))
    elif (z is not None):
        return(z.getText(strip = True))
    else:
        return("None")

    soup.find_all('div', class_=('content'))[1].find('div', class_="bbcode_container").find('div', class_='quote_container').getText(strip = True)

def process(page_id):
    baseurl = "https://bfriends.brigitte.de/foren/coronavirus/536653-kinder-familie-kita-schule-in-zeiten-von-corona-" + str(page_id) + ".html"
    req = Request(baseurl, data = None, headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    usernames = [i.getText() for i in soup.find_all(["a", "span"], {"class": "username"})]
    dates = [i.find(text=True, recursive = False) for i in soup.find_all("span", {"class": "date"})]
    times = [i.find('span', {"class": "time"}).getText() for i in soup.find_all("div", {"class": "posthead"})]

    meta = pandas.DataFrame({'user': usernames, 'date': dates, 'time': times})

    post = []
    
#     for i in soup.find_all(id = re.compile('post_message')):
# use below to cover cases deleted without post_message id
    for i in soup.find_all('div', class_=('content')):
        
        ix_ = i.find(id = re.compile('post_message'))
        ix = None
        if (ix_ is not None):
            ix = ix_.get('id')
    
        # block quoted text
        j = i.find_all('div', class_="bbcode_container") # can quote more than one m
        u, m = None, None
        if j:
            u = [get_quote_user(q) for q in j] # some does not have quoted user!!!
            m = [get_quote_msg(q) for q in j]
            [q.decompose() for q in j]

        emj = None
        im = i.find_all('img')
        if im:
            emj = [k.get('title') for k in im]
            [k.decompose() for k in im]

        # message removed emoji
        ms = i.getText(strip = True)

        post.append([ix] + [ms] + [emj] + [u] + [m])
    
    post = pandas.DataFrame(post, columns = ['mid', 'text', 'emoji', 'user_quoted', 'quoted_text'])
    
    return(meta.join(post))

In [ ]:
for pid in range(257, 1121+1):
    print(pid, end=".")
    op = process(pid)
    try:
        op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + str(pid) + ".csv")
    except:
        print("\nError in page"+str(pid)+"\n")
        pass

In [ ]:
# 257 emoji
op = process(257)
op.text[7] = op.text[7].replace('\ud83d\ude02', u'\U0001f602')
op.emoji[7] = 'JOY'
# https://charbase.com/1f602-unicode-face-with-tears-of-joy
# import emoji
# emoji.replace_emoji(op.text[7], replace='')
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '257' + ".csv")

In [ ]:
op = process(288)
op.text[3] = op.text[3].replace('\ud83d\ude23', u'\U0001f623 ')
op.emoji[3] = 'PERSEVERING'
# https://charbase.com/1f623-unicode-persevering-face
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '288' + ".csv")
op.text[3]

In [ ]:
op = process(499)

In [ ]:
op.text[9]

In [ ]:
op.text[3] = op.text[3].replace('\ud83d\ude23', u'\U0001f623 ')
op.emoji[3] = 'PERSEVERING'
# https://charbase.com/1f623-unicode-persevering-face
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '288' + ".csv")
op.text[3]